## 1. json to csv 

In [5]:
import numpy as np
import json
import pandas as pd

def convert_json_to_csv(json_file_path, output_csv_path):
    """
    JSON 데이터를 로드하여 DataFrame으로 변환 후 CSV 파일로 저장하는 함수.
    
    Parameters:
        json_file_path (str): JSON 파일 경로
        output_csv_path (str): 저장할 CSV 파일 경로
    """
    # JSON 파일 로드
    with open(json_file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    # DataFrame 변환
    df = pd.DataFrame(data)

    # 상담 텍스트(HS01, HS02, HS03만 합침) 추출
    df["text"] = df["talk"].apply(lambda x: " ".join([x["content"].get(k, "") for k in ["HS01", "HS02", "HS03"]]).strip())

    # 감정(E10~E69), 상황(S01~S13), 질병(D01~D02) 레이블 추출
    df["emotion"] = df["profile"].apply(lambda x: x["emotion"]["type"])  # 감정 ID (E10~E69)
    df["situation"] = df["profile"].apply(lambda x: x["emotion"]["situation"][0])  # 상황 ID (S01~S13)
    df["disease"] = df["profile"].apply(lambda x: x["emotion"]["situation"][1] if len(x["emotion"]["situation"]) > 1 else "D02")  # 질병 ID (D01 or D02)

    # 필요 없는 컬럼 제거
    df = df[["text", "emotion", "situation", "disease"]]

    # CSV 저장
    df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
    print(f"🙏 변환 완료: {output_csv_path}")

# 🔹 두 개의 JSON 파일 변환 실행
convert_json_to_csv("data/감성대화말뭉치(최종데이터)_Training.json", "data/train_dataset.csv")
convert_json_to_csv("data/감성대화말뭉치(최종데이터)_Validation.json", "data/test_dataset.csv")


✅ 변환 완료: data/train_dataset.csv
✅ 변환 완료: data/test_dataset.csv


## 2. 데이터프레임 샘플 출력

In [31]:
train_df = pd.read_csv('data/train_dataset.csv')
test_df = pd.read_csv('data/test_dataset.csv')

In [33]:
print(train_df.head())
print(test_df.head())

                                                text emotion situation disease
0  일은 왜 해도 해도 끝이 없을까? 화가 난다. 그냥 내가 해결하는 게 나아. 남들한...     E18       S06     D02
1  이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....     E18       S06     D02
2  회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...     E18       S06     D02
3  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...     E18       S06     D02
4  얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나. 상사인 나에게 ...     E18       S06     D02
                                                text emotion situation disease
0  이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...     E31       S06     D02
1  회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워. 나에게 너무 크...     E31       S06     D02
2  상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워. 무섭게 생겼는데도 업무를 보려...     E31       S06     D02
3  이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다. 첫 직장...     E31       S06     D02
4  직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼. 내가 낯가림이 심해서 친해질 수...     E31       S06     D02


In [35]:
print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51628 entries, 0 to 51627
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       51628 non-null  object
 1   emotion    51628 non-null  object
 2   situation  51628 non-null  object
 3   disease    51628 non-null  object
dtypes: object(4)
memory usage: 1.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6640 entries, 0 to 6639
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       6640 non-null   object
 1   emotion    6640 non-null   object
 2   situation  6640 non-null   object
 3   disease    6640 non-null   object
dtypes: object(4)
memory usage: 207.6+ KB
None


## **3. 학습 데이터 전처리**

In [37]:
import re
import pandas as pd

def preprocess_dataset(df, output_csv_path=None):
    """
    데이터프레임을 전처리하는 함수 (특수문자 제거 및 ID 매핑)

    Parameters:
        df (pd.DataFrame): 전처리할 데이터프레임
        output_csv_path (str, optional): 저장할 CSV 파일 경로 (기본값: None)

    Returns:
        pd.DataFrame: 전처리된 데이터프레임
    """
    # 🔹 (1) 텍스트 정제: 특수문자 제거 및 공백 정리
    def clean_text(text):
        text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)  # 한글, 영문, 숫자, 공백 제외 삭제
        text = re.sub(r"\s+", " ", text).strip()  # 연속된 공백 제거
        return text

    df["text"] = df["text"].apply(clean_text)

    # 🔹 (2) 감정, 상황, 질병 ID를 숫자로 변환
    emotion_mapping = {e: idx for idx, e in enumerate(sorted(df["emotion"].unique()))}
    situation_mapping = {s: idx for idx, s in enumerate(sorted(df["situation"].unique()))}
    disease_mapping = {"D01": 0, "D02": 1}  # 질병 여부를 0/1로 변환

    # 🔹 (3) 매핑 적용
    df["emotion"] = df["emotion"].map(emotion_mapping)
    df["situation"] = df["situation"].map(situation_mapping)
    df["disease"] = df["disease"].map(disease_mapping)

    # 🔹 (4) CSV 저장 (옵션)
    if output_csv_path:
        df.to_csv(output_csv_path, index=False, encoding="utf-8-sig")
        print(f"🙏 데이터 전처리 완료: {output_csv_path}")

    return df


In [39]:
# 데이터 로드
train_df = pd.read_csv("data/train_dataset.csv")
test_df = pd.read_csv("data/test_dataset.csv")

# 전처리 실행 및 CSV 저장
pre_train_df = preprocess_dataset(train_df, "data/pre_train_dataset.csv")
pre_test_df = preprocess_dataset(test_df, "data/pre_test_dataset.csv")


🙏 데이터 전처리 완료: data/pre_train_dataset.csv
🙏 데이터 전처리 완료: data/pre_test_dataset.csv


In [41]:
pre_train_df.head()

text  emotion  situation  \
0  일은 왜 해도 해도 끝이 없을까 화가 난다 그냥 내가 해결하는 게 나아 남들한테 부...        8          5   
1  이번 달에 또 급여가 깎였어 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나 최...        8          5   
2  회사에 신입이 들어왔는데 말투가 거슬려 그런 애를 매일 봐야 한다고 생각하니까 스트...        8          5   
3  직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜 일도 많은 데 정말 분하고 섭...        8          5   
4  얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나 상사인 나에게 먼...        8          5   

   disease  
0        1  
1        1  
2        1  
3        1  
4        1

In [43]:
pre_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51628 entries, 0 to 51627
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       51628 non-null  object
 1   emotion    51628 non-null  int64 
 2   situation  51628 non-null  int64 
 3   disease    51628 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.6+ MB


In [63]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader

# 데이터 로드
df = pd.read_csv("data/pre_train_dataset.csv")

# 🔹 (1) Huggingface KoBERT 모델 & 토크나이저 불러오기
model_name = "monologg/kobert"
tokenizer = BertTokenizer.from_pretrained(model_name)
kobert_model = BertModel.from_pretrained(model_name)

# 🔹 (2) 토큰화 함수 정의
def tokenize_text(text):
    encoded = tokenizer.encode_plus(
        text,
        max_length=64,  # 최대 64토큰까지 사용
        padding="max_length",
        truncation=True,
        return_tensors="pt"  # PyTorch Tensor 형식으로 반환
    )
    return encoded["input_ids"], encoded["attention_mask"]



# 🔹 (3) 데이터셋 클래스 정의
class EmotionDataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe["text"].tolist()
        self.emotions = dataframe["emotion"].tolist()
        self.situations = dataframe["situation"].tolist()
        self.diseases = dataframe["disease"].tolist()

    def __len__(self):
        return len(self.texts)

    # def __getitem__(self, idx):
    #     input_ids, attention_mask = tokenize_text(self.texts[idx])
    #     emotion_label = torch.tensor(self.emotions[idx], dtype=torch.long)
    #     situation_label = torch.tensor(self.situations[idx], dtype=torch.long)
    #     disease_label = torch.tensor(self.diseases[idx], dtype=torch.long)
    #     return input_ids, attention_mask, emotion_label, situation_label, disease_label

    def __getitem__(self, idx):
        input_ids, attention_mask = tokenize_text(self.texts[idx])
        return {
            "input_ids": input_ids.squeeze(0),
            "attention_mask": attention_mask.squeeze(0),
            "labels_emotion": torch.tensor(self.emotions[idx], dtype=torch.long),
            "labels_situation": torch.tensor(self.situations[idx], dtype=torch.long),
            "labels_disease": torch.tensor(self.diseases[idx], dtype=torch.long),
        }


# 데이터셋 및 데이터로더 생성
train_dataset = EmotionDataset(df)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# 첫 번째 배치 데이터 확인
for batch in train_loader:
    print("🔹 배치 데이터 샘플 확인:")
    print(f"input_ids 크기: {batch['input_ids'].shape}")
    print(f"attention_mask 크기: {batch['attention_mask'].shape}")
    print(f"labels_emotion 크기: {batch['labels_emotion'].shape}")
    print(f"labels_situation 크기: {batch['labels_situation'].shape}")
    print(f"labels_disease 크기: {batch['labels_disease'].shape}")
    break  # 첫 번째 배치만 확인


print("✅ Huggingface KoBERT 토큰화 및 데이터 준비 완료")


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

C:\Users\kj\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kj\.cache\huggingface\hub\models--monologg--kobert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

🔹 배치 데이터 샘플 확인:
input_ids 크기: torch.Size([16, 64])
attention_mask 크기: torch.Size([16, 64])
labels_emotion 크기: torch.Size([16])
labels_situation 크기: torch.Size([16])
labels_disease 크기: torch.Size([16])
✅ Huggingface KoBERT 토큰화 및 데이터 준비 완료
